# Lab setup

This tutorial walks through the process of setting up a lab and the related chemical inventory, vessels, tools etc.

First, we log into the server. In this case we are logging into a local instance of ESCALATE

In [1]:
import os
import sys
module_path = os.path.abspath(os.path.join('../../'))
if module_path not in sys.path:
    sys.path.append(module_path)
import escalateclient
import importlib

In [3]:
importlib.reload(escalateclient)
server_url = 'http://localhost:8000'
username = 'vshekar'
password = 'copperhead123'
client = escalateclient.ESCALATEClient(server_url, username, password)

Next, we set up organizations (they can be hierarchical). For example, we can create Neilson lab under University of Colorado

In [5]:
# Creating University of Colorado first
cu_info = { "description": "University of Colorado",
            "full_name": "University of Colorado at Boulder",
            "short_name": "CU",
            "address1": "Boulder, CO 80309",
            "address2": "",
            "city": "Boulder",
            "state_province": "CO",
            "zip": "80309",
            "country": "USA",
            "phone": '123456',
            "website_url": 'www.colorado.edu',
            }

cu_response = client.get_or_create(endpoint='organization', data=cu_info)
#cu_response = client.get(endpoint='organization', data=cu_info)
print(cu_response)

{'description': 'University of Colorado', 'full_name': 'University of Colorado at Boulder', 'short_name': 'CU', 'address1': 'Boulder, CO 80309', 'city': 'Boulder', 'state_province': 'CO', 'zip': '80309', 'country': 'USA', 'phone': '123456', 'website_url': 'www.colorado.edu'}
GET: OK. Found 0 results
POST: OK, returning new resource dict
[{'url': 'http://localhost:8000/api/organization/09e2eefd-1b84-458f-b6dd-6b027753f3a5/', 'uuid': '09e2eefd-1b84-458f-b6dd-6b027753f3a5', 'edocs': [], 'tags': [], 'notes': [], 'address1': 'Boulder, CO 80309', 'address2': '', 'city': 'Boulder', 'state_province': 'CO', 'zip': '80309', 'country': 'USA', 'phone': '123456', 'add_date': '2022-06-27T18:54:32.245719', 'mod_date': '2022-06-27T18:54:32.245751', 'description': 'University of Colorado', 'full_name': 'University of Colorado at Boulder', 'short_name': 'CU', 'website_url': 'www.colorado.edu', 'parent_path': None, 'internal_slug': 'university-of-colorado-at-boulder-cu', 'parent': None}]


In [6]:
# Then creating the Neilson Lab
neilson_lab_data = { 
    'description': 'Neilson Lab', 
    'address1': 'Boulder, CO 80309', 
    'address2': '', 
    'city': 'Boulder', 
    'state_province': 'CO', 
    'zip': '80309', 
    'country': 'USA', 
    'phone': '123456',
    'full_name': 'Neilson Lab', 
    'short_name': 'NL', 
    'website_url': 'www.colorado.edu', 
    'parent': cu_response[0]['url']}

nl_response = client.get_or_create(endpoint='organization', data=neilson_lab_data)

# Creating a person entry to identify the owner
# Note: If a user account is created, a person entry is automatically created for that user

neilson_data = {
    "first_name": "James",
    "middle_name": "",
    "last_name": "Neilson",
    "address1": "",
    "address2": "",
    "city": "",
    "state_province": "",
    "zip": "",
    "country": "",
    "phone": "",
    "email": "",
    "title": "",
    "suffix": "",
    "organization": cu_response[0]['url']
}

neilson_response = client.get_or_create('person', data=neilson_data)
neilson_actor = client.get('actor', data={'person': neilson_response[0]['url']})

{'description': 'Neilson Lab', 'address1': 'Boulder, CO 80309', 'city': 'Boulder', 'state_province': 'CO', 'zip': '80309', 'country': 'USA', 'phone': '123456', 'full_name': 'Neilson Lab', 'short_name': 'NL', 'website_url': 'www.colorado.edu', 'parent': '09e2eefd-1b84-458f-b6dd-6b027753f3a5'}
GET: OK. Found 0 results
POST: OK, returning new resource dict
{'first_name': 'James', 'last_name': 'Neilson', 'organization': '09e2eefd-1b84-458f-b6dd-6b027753f3a5'}
GET: OK. Found 0 results
POST: OK, returning new resource dict
{'person': '1074acc8-4beb-4638-8801-05324bd4c9c2'}
GET: OK. Found 1 results


Next, we set up an inventory for the lab

In [7]:
# Get value, get always returns a list, even if there is only one element to be returned
active_status = client.get_or_create(endpoint='status', data={'description': 'active'})

# Inventory details
nl_inventory = {
    "description": "Nielson Lab Inventory",
    "status": active_status[0]['url'], # Indicates active status
    "actor": neilson_actor[0]['url'],
    "owner": neilson_actor[0]['url'], # Indicates James Neilson as owner
    "operator": neilson_actor[0]['url'], # Indicates James Neilson as operator
    "lab": neilson_actor[0]['url'] # Associates inventory with Neilson Lab
}

nl_inventory_response = client.get_or_create(endpoint='inventory', data=nl_inventory)

{'description': 'active'}
GET: OK. Found 1 results
{'description': 'Nielson Lab Inventory', 'status': 'f105cba7-c77b-434f-9126-7f0908995fde', 'actor': '26969748-2725-465e-95e1-17f76b50a3a0', 'owner': '26969748-2725-465e-95e1-17f76b50a3a0', 'operator': '26969748-2725-465e-95e1-17f76b50a3a0', 'lab': '26969748-2725-465e-95e1-17f76b50a3a0'}
GET: OK. Found 0 results
POST: OK, returning new resource dict


Then we can add chemicals to the inventory - the easiest way is to load from a .csv file. At the bare minimum, all we need is a description for each chemical.

In [79]:
import pandas as pd
df = pd.read_csv('MetalChalcogenideRxns.csv')
#Add material to db
data = {
    'description': '',
    'consumable': True,
    'material_class': 'model',
}
for chemical in df['Product expected']:
    data['description'] = chemical
    client.post('material', data=data)

POST: OK, returning new resource dict
POST: OK, returning new resource dict
POST: OK, returning new resource dict
POST: OK, returning new resource dict
POST: OK, returning new resource dict
POST: OK, returning new resource dict
POST: OK, returning new resource dict
POST: OK, returning new resource dict
POST: OK, returning new resource dict
POST: OK, returning new resource dict
POST: OK, returning new resource dict
POST: OK, returning new resource dict
POST: OK, returning new resource dict
POST: OK, returning new resource dict
POST: OK, returning new resource dict
POST: OK, returning new resource dict
POST: OK, returning new resource dict
POST: OK, returning new resource dict
POST: OK, returning new resource dict
POST: OK, returning new resource dict
POST: OK, returning new resource dict
POST: OK, returning new resource dict
POST: OK, returning new resource dict
POST: OK, returning new resource dict
POST: OK, returning new resource dict
POST: OK, returning new resource dict


In [80]:
for i, row in df.iterrows():
    material_name = row['Product expected']
    # Check that material is in db (add it if it isn't)
    material_data = {'description': material_name, 
                        'consumable': True,
                        'material_class':'model'}
    material_response = client.get_or_create('material', data=material_data)
    # Add material to inventory
    im_data = {
        "description": material_name,
        "phase": "solid",
        "inventory": nl_inventory_response[0]['url'],
        "material": material_response[0]['url']
    }
    im_response = client.get_or_create('inventory-material', data=im_data)

{'description': 'FeSe', 'consumable': True, 'material_class': 'model'}
GET: OK. Found 4 results
{'description': 'FeSe', 'phase': 'solid', 'inventory': 'a2d51de7-2e8a-468c-b0d4-ffc1e7b2da01', 'material': 'ae40e9e1-6584-499c-bf58-4a92e033da82'}
GET: OK. Found 1 results
{'description': 'FeS', 'consumable': True, 'material_class': 'model'}
GET: OK. Found 4 results
{'description': 'FeS', 'phase': 'solid', 'inventory': 'a2d51de7-2e8a-468c-b0d4-ffc1e7b2da01', 'material': 'c8303c05-19a8-4f5e-b540-58049ae186e6'}
GET: OK. Found 1 results
{'description': 'FeTe2', 'consumable': True, 'material_class': 'model'}
GET: OK. Found 4 results
{'description': 'FeTe2', 'phase': 'solid', 'inventory': 'a2d51de7-2e8a-468c-b0d4-ffc1e7b2da01', 'material': 'e7c04161-228f-4e68-8b01-1e28d73a9136'}
GET: OK. Found 1 results
{'description': 'Co9Se8', 'consumable': True, 'material_class': 'model'}
GET: OK. Found 4 results
{'description': 'Co9Se8', 'phase': 'solid', 'inventory': 'a2d51de7-2e8a-468c-b0d4-ffc1e7b2da01', '

 But depending on our intent for use of ESCALATE we might want to add more information for each material. For best results it is recommended to associate a material type with each material, so that materials can be sorted accordingly. It's easiest to just include the material type as part of our .csv 

 Below we demonstrate with a different .csv file - note that we add materials to the database and to the inventory in the same "for" loop this time around

In [81]:
chem_list = pd.read_csv('Chemicals List.csv')
chem_list = chem_list.fillna('')
chem_list['Material type'] = chem_list['Material type'].str.replace('Gas', 'flux')

for i, row in chem_list.iterrows():
    chemical_name = row['Chemical Name']
    material_types = row['Material type'].lower().split(',')
    mt_responses = []
    for mt in material_types:
        mt_responses.append(client.get_or_create('material-type', data={'description': mt})[0]) 
    # Add material to db
    material_data = {'description': chemical_name, 'material_type': [mtr['url'] for mtr in mt_responses], 'material_class':'model'}
    material_response = client.get_or_create('material', data=material_data)
    # Add material to inventory
    if row['Inventory Name']:
        description = f"{chemical_name} {row['Inventory Name']}"
    else:
        description = chemical_name
    im_data = {
        "description": description,
        "part_no": f"{row['CAS Num']}",
        "phase": row['Phase'].lower(),
        "inventory": nl_inventory_response[0]['url'],
        "material": material_response[0]['url']
    }
    im_response = client.get_or_create('inventory-material', data=im_data)

{'description': 'flux'}
GET: OK. Found 1 results
{'description': 'Argon', 'material_type': ['http://localhost:8000/api/material-type/d206685a-122c-4c68-b7ab-328e4cbb5175/'], 'material_class': 'model'}
GET: OK. Found 0 results
POST: OK, returning new resource dict
[{'url': 'http://localhost:8000/api/material/a54aab17-4bae-4c94-9477-2e7bccc8bbcf/', 'uuid': 'a54aab17-4bae-4c94-9477-2e7bccc8bbcf', 'edocs': [], 'tags': [], 'notes': [], 'property': [], 'add_date': '2022-06-27T20:36:17.484504', 'mod_date': '2022-06-27T20:36:17.484547', 'description': 'Argon', 'consumable': None, 'material_class': 'model', 'internal_slug': 'argon-3', 'status': None, 'actor': None, 'identifier': [], 'material_type': ['http://localhost:8000/api/material-type/d206685a-122c-4c68-b7ab-328e4cbb5175/']}]
{'description': 'organic'}
GET: OK. Found 1 results
{'description': 'Benzene', 'material_type': ['http://localhost:8000/api/material-type/7b0228e0-96eb-4c57-b43f-4e940020096d/'], 'material_class': 'model'}
GET: OK. F

Optionally we might want to include identifiers for machine learning purposes, like SMILES strings for the chemicals, or properties such as molecular weight/density, for calculations.

In [84]:
# Get or create property definition 
property_data={'description':'density', 'property_def_class': 'intrinsic'}
property_template_response = client.get_or_create('property-template', data=property_data)

#Add property to materials 
for i, material in enumerate(df['Product expected']):
    material_data={'description': material}
    material_response = client.get('material', data=material_data)
    density = df['avg density'][i]
    
    data = {
    'value': {'value': float(density), 'unit': 'g/ml', 'type': 'num'},
    'material': material_response[0]['url'],
    'template': property_template_response[0]['url']
}
    client.post('property', data)

{'description': 'density', 'property_def_class': 'intrinsic'}
GET: OK. Found 1 results
{'description': 'FeSe'}
GET: OK. Found 4 results
POST: FAILED, returning response object
Status 500: Internal Server Error <!DOCTYPE html>
<html lang="en">
<head>
  <meta http-equiv="content-type" content="text/html; charset=utf-8">
  <meta name="robots" content="NONE,NOARCHIVE">
  <title>KeyError
          at /api/property/</title>
  <style type="text/css">
    html * { padding:0; margin:0; }
    body * { padding:10px 20px; }
    body * * { padding:0; }
    body { font:small sans-serif; background-color:#fff; color:#000; }
    body>div { border-bottom:1px solid #ddd; }
    h1 { font-weight:normal; }
    h2 { margin-bottom:.8em; }
    h3 { margin:1em 0 .5em 0; }
    h4 { margin:0 0 .5em 0; font-weight: normal; }
    code, pre { font-size: 100%; white-space: pre-wrap; }
    table { border:1px solid #ccc; border-collapse: collapse; width:100%; background:white; }
    tbody td, tbody th { vertical-align

For clarity, here is a breakdown of the various database fields that can be associated with chemicals, at the material level and the inventory level. Again, only descriptions are required, but including other fields makes it easier to work with ESCALATE

**Material fields**
* material type (e.g. "organic")
* material identifier (e.g. "molecular formula")
* material property (e.g. "density")

**Inventory material fields**
* phase (e.g. "solid")
* part no. (e.g. "CAS Number")

The fields at the material level link to other database tables. for instance, there are database entries for different material types. that means that these fields from other tables must be accessed first and then associated with the material. in the event that a particular entry does not exist in the associated table, it must be created.

At the inventory material level, the listed fields are string descriptions

To load your data most efficiently, we recommend compiling a .csv file that contains all the database fields you will make use of and then following the above procedure to create an entry at the material and at the inventory material level


Like chemicals, vessels can be added to the database from a .csv file. We can also post data for individual vessels

In [52]:
for v in client.get('vessel'):
    if not v['parent']:
        print(v['description'])

{}
GET: OK. Found 945 results
biorad_96_wellplate_200ul_pcr
corning_12_wellplate_6.9ml_flat
corning_24_wellplate_3.4ml_flat
corning_48_wellplate_1.6ml_flat
corning_6_wellplate_16.8ml_flat
corning_6_wellplate_16.8ml_flat
nest_96_wellplate_100ul_pcr_full_skirt
nest_96_wellplate_200ul_flat
nest_96_wellplate_2ml_deep
usascientific_96_wellplate_2.4ml_deep
Symyx_96_well_0003
Plate: 24 Well
Plate: 48 Well
96 Well Plate well
Generic Vessel


Next, we check to make sure the database contains the property definitions, action definitions and parameters we'll need. This ensures that we have all the pieces necessary to create experiment templates seamlessly - but we can always go back and add missing definitions later during the template creation process if we forget any

In [59]:
#Print a list of existing action definitions
for action_def in client.get('action-def'):
    print(action_def['description'])

{}
GET: OK. Found 7 results
heat
heat_stir
stir
dispense
bring_to_temperature
dispense_solid
dwell


In [69]:
#Add new action definition
params = client.get_or_create('parameter-def', data={'description': 'temperature'})

client.post('action-def', data={'description': 'cool1', 'parameter_def': params[0]['url']})

{'description': 'temperature'}
GET: OK. Found 1 results
http://localhost:8000/api/parameter-def/9bd72d5c-1677-44b7-b870-371f42b5c102/
POST: FAILED, returning response object
Status 400: Bad Request {"parameter_def":["Expected a list of items but got type \"str\"."]}


<Response [400]>